In [2]:
%pip install boto3
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
import boto3
import json
import base64


In [5]:


class Chat:

    def __init__(self, model_id, bedrock_runtime_client):
        self.model_id = model_id
        self.bedrock_runtime_client = bedrock_runtime_client
        self.payload = {
            "messages": [],
            "max_tokens": 10000,
            "anthropic_version": "bedrock-2023-05-31"
        }



    def add_user_message(self, message):
        self.payload["messages"].append({
                                        "role": "user",
                                        "content": [
                                            {
                                                "type": "text",
                                                "text": message
                                            }
                                        ]
                                    })

    def add_user_message_image(self, message, encoded_image):
        self.payload["messages"].append({
                                        "role": "user",
                                        "content": [
                                                    {
                                                        "type": "image",
                                                        "source": {
                                                            "type": "base64",
                                                            "media_type": "image/jpeg",
                                                            "data": encoded_image,
                                                            # "temperature": 1,
                                                        }
                                                    },
                                                    {
                                                        "type": "text",
                                                        "text": message,
                                                        # "temperature": 1,
                                                    }
                                                ]
                                    })


    def generate(self):
        response = self.bedrock_runtime_client.invoke_model(
            modelId=self.model_id,
            contentType="application/json",
            body=json.dumps(self.payload)
        )

        # now we need to read the response. It comes back as a stream of bytes so if we want to display the response in one go we need to read the full stream first
        # then convert it to a string as json and load it as a dictionary so we can access the field containing the content without all the metadata noise
        output_binary = response["body"].read()
        output_json = json.loads(output_binary)
        output = output_json["content"][0]["text"]


        self.payload["messages"].append(
            {
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": output
                    }
                ]
            }
        )

        return output


def load_image(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

In [6]:
# Load environment variables from .env file (they are loaded as strings)
load_dotenv()

# Load keys from environment variables
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

if not AWS_ACCESS_KEY_ID or not AWS_SECRET_ACCESS_KEY:
    raise ValueError("Please set the AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY environment variables")

In [8]:
# Define the aws runtime and model

bedrock_runtime_client = boto3.client(
    'bedrock-runtime',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name="us-west-2"
)

model_id = "anthropic.claude-3-haiku-20240307-v1:0"

In [ ]:
prompt = "Give me a description of the image."
image_path = "content/istockphoto-530427077-612x612.jpg"
chat = Chat(model_id, bedrock_runtime_client)

chat.add_user_message_image(prompt, load_image(image_path))
chat.generate()

In [ ]:
chat.add_user_message("Give me a list of objects.")
chat.generate()